# Prueba de sistemas

[ ] - Prueba de data loader
[ ] - Prueba de modelo

In [1]:
import glob

import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

import sklearn
import rdkit
import torch

from rdkit.Chem.rdmolfiles import MolFromSmiles
from rdkit.Chem            import AllChem

import libs.lstm_mol as lstm_mol

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.get_device_name(device))

NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
data_dir = 'datasets/full'

omdb_df = pd.read_csv(f'{data_dir}/omdb_smile_data_set.csv')

print(omdb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11368 entries, 0 to 11367
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CODID   11368 non-null  int64  
 1   SMILE   11368 non-null  object 
 2   bgs     11368 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 266.6+ KB
None


In [3]:
model = lstm_mol.LSTM_MOL(5).to(device)

for i in range(3):
	smile = omdb_df['SMILE'][i]
	x = torch.tensor([float(ord(char)) for char in smile]).unsqueeze(0).to(device)
	# print(x)
	out = model(x)

	print(f'{smile}:\n   ->{out}')

[Mo]1234([P]5(OC)N(C)CCN5C)(P5N(C)CCN5C)([cH]5[cH]1[cH]2[cH]3[cH]45)C#[O]:
   ->tensor([-0.0460], device='cuda:0', grad_fn=<ViewBackward0>)
Clc1c(Cl)ccc(c1)[C@H](Cl)[C@H](Cl)[C@H](O)C(=O)OC.Clc1c(Cl)ccc(c1)[C@@H](Cl)[C@@H](Cl)[C@@H](O)C(=O)OC:
   ->tensor([-0.0429], device='cuda:0', grad_fn=<ViewBackward0>)
O[C@H]1[C@H]2[C@H](NCc3c4OCOc4ccc23)c2c(C1)cc1OCOc1c2.O[C@@H]1[C@@H]2[C@@H](NCc3c4OCOc4ccc23)c2c(C1)cc1OCOc1c2:
   ->tensor([-0.0450], device='cuda:0', grad_fn=<ViewBackward0>)
